In [1]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')
#import matplotlib.pyplot as plt

# https://duckdb.org/docs/guides/python/jupyter
#SQL in Pandas
import duckdb
import sqlalchemy
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql sqlite:///DB/data.db


# Annotations
## Ideas/Questions

- What is the Gross Profit
- Main Insights like AVG Turnover, AVG Units, AVG Cost of sales OVER ALL
- Best(Profit) Regions OVER ALL, OVER TIME, OVER MONTHS OVER THE YEAR -> VIZ
- Best(Profit) Products OVER ALL, OVER TIME, OVER MONTHS OVER THE YEAR -> VIZ
- Best(Profit) Month OVER ALL, OVER TIME -> VIZ
- Profit, Cost of Sale per Product, Profit Margin, Best Margin of Product OVER TIME
- Versions?
- VIZ-Exploration:
  - Distribution
    - HISTOGRAM of X:Units, Y:Proft per Product
    - SCATTERPLOT for Regions
    - SCATTERPLOT for 
- What is the relation of UNITS, COST OF SALES and GROSS PROFIT, how can they all be negative? Logistics?
- Experimentation:
  - Quiz for best Products

## Data Import SQL
    - create SQLite DB, read xlsx, convert to df and export it sales_import table
    - Cheatsheet: https://www.cockroachlabs.com/blog/sql-cheat-sheet/#create-an-index-based-on-a-column
    - Turnover-Ratios: https://www.wallstreetmojo.com/turnover-ratios-formula/
    - DuckDB to integrate SQL in Jupyter Notebook: # https://duckdb.org/docs/guides/python/jupyter

In [2]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///DB/data.db")
data = pd.read_excel('./Data/data_export_sales.xlsx')
data.to_sql("sales_import", con=engine, if_exists="replace")

/home/stubertus/.local/share/virtualenvs/bichallenge-BMVbwoS7/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


201110

In [6]:
%reload_ext sql

In [1]:
#negData = data.loc[(data['Turnover'] < 0) & (data['Units'] < 0) & (data['Cost of Sales'] < 0)]
#negData.style.highlight_between('Units', left=data['Units'].min(), right=0, axis=0, color='red')

In [4]:
#Versions == 'Actual'

# Bubble Treemap
https://jgoertler.com/bubble-treemaps/

# SQL Import to DF

In [3]:
%%sql col <<

SELECT * FROM sales_import

Returning data to local variable col


In [4]:
col

,index,Products,Regions,Months,Years,Versions,Units,Cost of Sales,Turnover,Gross Profit
0,0,Desktop High XL,Austria,Apr,2014,Actual,4730.030000,20156.200000,22900.400000,0.0
1,1,Desktop High XL,Austria,Apr,2014,All Versions,741.340000,2900.600000,3553.300000,0.0
2,2,Desktop High XL,Austria,Apr,2014,Budget,3988.690000,17255.600000,19347.100000,0.0
3,3,Desktop High XL,Austria,Apr,2014,Variance,741.340000,2900.600000,3553.300000,0.0
4,4,Desktop High XL,Austria,Apr,2015,Actual,4966.531500,21164.010000,24045.420000,0.0
...,...,...,...,...,...,...,...,...,...,...
201105,201105,Wireless Mouse XT,United Kingdom,Year,2020,Variance,-2.105263,0.000000,0.000000,0.0
201106,201106,Wireless Mouse XT,United Kingdom,Year,All Years,Actual,17877.417666,68517.727090,75290.998183,0.0
201107,201107,Wireless Mouse XT,United Kingdom,Year,All Years,All Versions,-13662.605007,-42549.122372,-56026.620653,0.0
201108,201108,Wireless Mouse XT,United Kingdom,Year,All Years,Budget,31540.022673,111066.849461,131317.618837,0.0


## Information zum Datensatz
| Spalte            | Inhalt                         |
|-------------------|--------------------------------|
| Products          | Hardware und Peripherie, welche das Unternehmen verkauft |
| Regions           | Regionen, in welchen die Produkte verkauft werden |
| Months            | Monatssummen, Quartal-Summen und SUmmen aller Produkte |
| Years             | Summen für die Jahre 2014, 2015, 2016, 2017, 2019 und für das ganze Jahr|
| Versions          | (All Versions OR Variance) + Budget = Actual >> Wir nutzen nur Actual, weil Definitionen nicht klar |
| Units             | Monatssummen, Quartal-Summen und SUmmen aller Produkte |
| Cost of Sales     | Monatssummen, Quartal-Summen und SUmmen aller Produkte |
| Turnover          | Monatssummen, Quartal-Summen und SUmmen aller Produkte |
| Gross Profit       | Monatssummen, Quartal-Summen und SUmmen aller Produkte |

 


In [74]:
%%sql monthSales <<

SELECT *, "Turnover" - "Cost of Sales" as "GP", (("Turnover" - "Cost of Sales")) / ("Turnover") * 100 AS "GrossMargin"
FROM sales_import
WHERE Months in ('Jan','Feb','Mar','Apr','Mai','Jun','Jul','Aug','Sep','Oct','Nov','Dec') AND Years <> 'All Years' AND Versions = 'Actual'

Returning data to local variable monthSales


In [75]:
%%sql allSales <<

SELECT *, "Turnover" - "Cost of Sales" as "GP", (("Turnover" - "Cost of Sales")) / ("Turnover") * 100 AS "GrossMargin"
FROM sales_import
WHERE Years = 'All Years' AND Months = 'Year' AND Versions = 'Actual'

Returning data to local variable allSales


In [94]:
%%sql bestRegions <<

SELECT Regions, SUM(Turnover) AS 'Turnover', SUM(Units) as 'Units', SUM("Turnover" - "Cost of Sales") AS "GP", ((SUM("Turnover" - "Cost of Sales")) / (SUM("Turnover"))) * 100 AS "GrossMargin"
from sales_import
WHERE Years = 'All Years' AND Months = 'Year' AND Versions = 'Actual'
GROUP BY Regions
ORDER BY cast(SUM(Turnover) AS FLOAT) DESC

Returning data to local variable bestRegions


In [95]:
%%sql bestProducts <<

SELECT Products, SUM(Turnover) AS 'Turnover', SUM(Units) as 'Units', SUM("Turnover" - "Cost of Sales") AS "GP", ((SUM("Turnover" - "Cost of Sales")) / (SUM("Turnover"))) * 100 AS "GrossMargin"
from sales_import
WHERE Years = 'All Years' AND Months = 'Year' AND Versions = 'Actual'
GROUP BY Products
ORDER BY cast(SUM(Turnover) AS FLOAT) DESC

Returning data to local variable bestProducts


In [96]:
bestRegions.sort_values(by='GrossMargin', ascending=False)

,Regions,Turnover,Units,GP,GrossMargin
4,Spain,2.324009e+07,5.381615e+06,2.755681e+06,11.857444
8,Denmark,1.384573e+07,3.160679e+06,1.531853e+06,11.063725
14,Norway,6.746575e+06,1.530765e+06,7.417394e+05,10.994310
5,Austria,1.912670e+07,4.379840e+06,2.102277e+06,10.991322
2,United Kingdom,2.675621e+07,6.239173e+06,2.937501e+06,10.978764
10,Czech Republic,1.317355e+07,3.047783e+06,1.442659e+06,10.951180
12,Portugal,1.018454e+07,2.371030e+06,1.088609e+06,10.688841
7,Poland,1.554768e+07,3.701037e+06,1.655277e+06,10.646457
6,Belgium,1.560840e+07,3.564655e+06,1.654493e+06,10.600021
16,Ireland,5.493386e+06,1.271220e+06,5.787008e+05,10.534501


In [97]:
allSales.groupby('Regions').count()

,index,Products,Months,Years,Versions,Units,Cost of Sales,Turnover,Gross Profit,GP,GrossMargin
Regions,,,,,,,,,,,
Austria,24,24,24,24,24,24,24,24,24,24,24
Belgium,24,24,24,24,24,24,24,24,24,24,24
Czech Republic,24,24,24,24,24,24,24,24,24,24,24
Denmark,24,24,24,24,24,24,24,24,24,24,24
Finland,23,23,23,23,23,23,23,23,23,23,23
France,24,24,24,24,24,24,24,24,24,24,24
Germany,24,24,24,24,24,24,24,24,24,24,24
Ireland,23,23,23,23,23,23,23,23,23,23,23
Italy,24,24,24,24,24,24,24,24,24,24,24


In [114]:
allSales.groupby('Regions')

index            19
Units            19
Cost of Sales    19
Turnover         19
Gross Profit     19
GP               19
GrossMargin      19
dtype: int64

In [99]:
allSales.groupby('Regions').agg({'Turnover':'sum','Units':'sum','GP':'sum','GrossMargin':'mean'}).sort_values('GrossMargin',ascending=False)

,Turnover,Units,GP,GrossMargin
Regions,,,,
Austria,1.912670e+07,4.379840e+06,2.102277e+06,11.454735
Spain,2.324009e+07,5.381615e+06,2.755681e+06,11.354750
Belgium,1.560840e+07,3.564655e+06,1.654493e+06,11.296027
Czech Republic,1.317355e+07,3.047783e+06,1.442659e+06,11.042928
Portugal,1.018454e+07,2.371030e+06,1.088609e+06,10.930383
Norway,6.746575e+06,1.530765e+06,7.417394e+05,10.718290
Denmark,1.384573e+07,3.160679e+06,1.531853e+06,10.380012
Slovakia,3.041124e+06,6.988459e+05,3.009251e+05,10.325851
Finland,6.844134e+06,1.621948e+06,6.927324e+05,10.279813


In [32]:
bestProductsPerRegion[bestProductsPerRegion['Regions'] == 'Germany'].GrossMargin.mean()

9.096001767106863

In [10]:
allSales.groupby('Products').sum().sort_values(by='Turnover', ascending=False)

NameError: name 'allSales' is not defined

## Sums

In [186]:
allSales = allSales[allSales['Versions'] == 'Actual']

In [190]:
8.609179e+06

8609179.0

In [188]:
allSales.groupby("Products").sum()

,index,Units,Cost of Sales,Turnover,Gross Profit,GP
Products,,,,,,
Desktop High XL,87152,1.969977e+06,7.651184e+06,8.609179e+06,0.0,9.579943e+05
Desktop High XQ,252366,1.462116e+06,5.645716e+06,6.331598e+06,0.0,6.858822e+05
Desktop L,416788,4.490466e+06,1.688419e+07,1.898460e+07,0.0,2.100413e+06
Desktop Pro,583146,4.151034e+06,1.564791e+07,1.785606e+07,0.0,2.208151e+06
Desktop Pro XL,747047,2.948819e+06,1.154470e+07,1.281641e+07,0.0,1.271712e+06
Keyboard GT,804098,4.651626e+05,1.796927e+06,2.016820e+06,0.0,2.198924e+05
Keyboard XX,1041209,1.634231e+05,6.180421e+05,6.822705e+05,0.0,6.422840e+04
Laptop Mouse S,1183633,4.601118e+05,1.809284e+06,1.997140e+06,0.0,1.878564e+05
Notebook GT,1337964,5.617414e+06,2.174949e+07,2.437935e+07,0.0,2.629854e+06


In [11]:
monthSales = monthSales[monthSales['Versions'] == 'Actual']

In [12]:
monthSales[monthSales['Years'] == '2018']['Months'].unique()

array(['Apr', 'Aug', 'Dec', 'Jan', 'Jul', 'Jun', 'Mar', 'Oct', 'Sep',
       'Feb', 'Nov'], dtype=object)

In [12]:
allSum = allSales.sum()
allAvg = allSales.mean()
monthSum = monthSales.sum()
monthAvg = monthSales.mean()


/tmp/ipykernel_6900/1650940956.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  allAvg = allSales.mean()
/tmp/ipykernel_6900/1650940956.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  monthAvg = monthSales.mean()


In [13]:
allSum

index                                                     45849718
Products         Desktop High XLDesktop High XLDesktop High XLD...
Regions          AustriaBelgiumCzech RepublicDenmarkFinlandFran...
Months           YearYearYearYearYearYearYearYearYearYearYearYe...
Years            All YearsAll YearsAll YearsAll YearsAll YearsA...
Versions         ActualActualActualActualActualActualActualActu...
Units                                               63830573.88037
Cost of Sales                                     246165738.491058
Turnover                                           275136536.33202
Gross Profit                                                   0.0
GP                                                 28970797.840962
dtype: object

In [51]:
monthSum

index                                                   2218858499
Products         Desktop High XLDesktop High XLDesktop High XLD...
Regions          AustriaAustriaAustriaAustriaAustriaAustriaAust...
Months           AprAprAprAprAprAugAugAugAugDecDecDecDecDecJanJ...
Years            2014201520162017201820142015201620182014201520...
Versions         ActualActualActualActualActualActualActualActu...
Units                                              58931500.285589
Cost of Sales                                     224248160.429912
Turnover                                          250702464.647641
Gross Profit                                                   0.0
GP                                                 26454304.217728
dtype: object

## Cleaning & Checking Data

In [14]:
allSales.isnull().sum()

index            0
Products         0
Regions          0
Months           0
Years            0
Versions         0
Units            0
Cost of Sales    0
Turnover         0
Gross Profit     0
GP               0
dtype: int64

In [15]:
allSales.duplicated().sum()


0

# Datetime Conversion

In [16]:
monthSales['Datetime'] = monthSales['Months'] + '.' + monthSales['Years']
monthSales['Datetime'] = pd.to_datetime(monthSales['Datetime'], format="%b.%Y")

# Profit per Region

## Enhancing Data, Gross Margin, GDP, Population, gdpPerPerson
https://corporatefinanceinstitute.com/resources/knowledge/accounting/gross-profit/

### Add populatoin to country

- https://github.com/ajturner/acetate/blob/master/places/Countries-Europe.csv
- https://github.com/datasets/gdp/blob/master/data/gdp.csv

In [131]:
pop_countries = pd.read_csv('./Data/countries.csv')
pop_countries = pop_countries[['name','population','ISO alpha 3']]

In [132]:
gdp_countries = pd.read_csv('Data/gdp.csv')
gdp_countries = gdp_countries[['Country Name','Country Code','Value', "Year"]]


In [133]:
gdp_countries = gdp_countries[gdp_countries['Year']==2016]

In [135]:
bestRegions.sort_values(by='GrossMargin', ascending=False)

,Regions,Turnover,Units,GP,GrossMargin
4,Spain,2.324009e+07,5.381615e+06,2.755681e+06,11.857444
8,Denmark,1.384573e+07,3.160679e+06,1.531853e+06,11.063725
14,Norway,6.746575e+06,1.530765e+06,7.417394e+05,10.994310
5,Austria,1.912670e+07,4.379840e+06,2.102277e+06,10.991322
2,United Kingdom,2.675621e+07,6.239173e+06,2.937501e+06,10.978764
10,Czech Republic,1.317355e+07,3.047783e+06,1.442659e+06,10.951180
12,Portugal,1.018454e+07,2.371030e+06,1.088609e+06,10.688841
7,Poland,1.554768e+07,3.701037e+06,1.655277e+06,10.646457
6,Belgium,1.560840e+07,3.564655e+06,1.654493e+06,10.600021
16,Ireland,5.493386e+06,1.271220e+06,5.787008e+05,10.534501


In [155]:
regions_pop = bestRegions.merge(pop_countries, left_on="Regions", right_on="name", how="left")
regions_gdp = regions_pop.merge(gdp_countries, left_on="Regions", right_on="Country Name", how="left")
regions_gdp.rename(columns={'Value':'GDP'}, inplace=True)
regions_gdp['Regions'] = regions_gdp['Regions'].apply(lambda x: 'Slovak Republic' if x == "Slovakia" else x)
regions_gdp['gdpPerPerson'] = regions_gdp['GDP'] / regions_gdp['population']


In [157]:
regions_gdp

,Regions,Turnover,Units,GP,GrossMargin,name,population,ISO alpha 3,Country Name,Country Code,GDP,Year,gdpPerPerson
0,Germany,3.050744e+07,7.262847e+06,3.047059e+06,9.987921,Germany,82369000,DEU,Germany,DEU,3.477796e+12,2016.0,42222.150014
1,France,2.800211e+07,6.450927e+06,2.796250e+06,9.985855,France,64768389,FRA,France,FRA,2.465454e+12,2016.0,38065.698612
2,United Kingdom,2.675621e+07,6.239173e+06,2.937501e+06,10.978764,United Kingdom,62348447,GBR,United Kingdom,GBR,2.647899e+12,2016.0,42469.360217
3,Italy,2.349302e+07,5.383176e+06,2.239569e+06,9.532913,Italy,58145000,ITA,Italy,ITA,1.858913e+12,2016.0,31970.301211
4,Spain,2.324009e+07,5.381615e+06,2.755681e+06,11.857444,Spain,46505963,ESP,Spain,ESP,1.237255e+12,2016.0,26604.223197
5,Austria,1.912670e+07,4.379840e+06,2.102277e+06,10.991322,Austria,8205000,AUT,Austria,AUT,3.908000e+11,2016.0,47629.493132
6,Belgium,1.560840e+07,3.564655e+06,1.654493e+06,10.600021,Belgium,10403000,BEL,Belgium,BEL,4.679557e+11,2016.0,44982.765531
7,Poland,1.554768e+07,3.701037e+06,1.655277e+06,10.646457,Poland,38500000,POL,Poland,POL,4.713644e+11,2016.0,12243.231395
8,Denmark,1.384573e+07,3.160679e+06,1.531853e+06,11.063725,Denmark,5484000,DNK,Denmark,DNK,3.068997e+11,2016.0,55962.737675
9,Switzerland,1.377640e+07,3.217742e+06,1.363649e+06,9.898443,Switzerland,7581000,CHE,Switzerland,CHE,6.688513e+11,2016.0,88227.317800


In [179]:
bestRegions

,Regions,Turnover,Units,GP,GrossMargin
0,Germany,3.050744e+07,7.262847e+06,3.047059e+06,9.987921
1,France,2.800211e+07,6.450927e+06,2.796250e+06,9.985855
2,United Kingdom,2.675621e+07,6.239173e+06,2.937501e+06,10.978764
3,Italy,2.349302e+07,5.383176e+06,2.239569e+06,9.532913
4,Spain,2.324009e+07,5.381615e+06,2.755681e+06,11.857444
5,Austria,1.912670e+07,4.379840e+06,2.102277e+06,10.991322
6,Belgium,1.560840e+07,3.564655e+06,1.654493e+06,10.600021
7,Poland,1.554768e+07,3.701037e+06,1.655277e+06,10.646457
8,Denmark,1.384573e+07,3.160679e+06,1.531853e+06,11.063725
9,Switzerland,1.377640e+07,3.217742e+06,1.363649e+06,9.898443


In [178]:
selection = alt.selection_multi(fields=['Regions'], bind='legend')


alt.Chart(regions_gdp).mark_point().encode(
    x= alt.X('Turnover', axis=alt.Axis(title='Turnover')),
    y= alt.Y('GrossMargin', axis=alt.Axis(title='Gross Margin')),
    size='gdpPerPerson',
    color=alt.Color('Regions', sort='-x'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
).add_selection(selection)

alt.Chart(...)

In [152]:
regions_gdp.groupby(['Regions','gdpPerPerson']).mean().sort_values(by=['GrossMargin'], ascending=False)

,,Turnover,Units,GP,GrossMargin,population,GDP,Year,MarginGDP
Regions,gdpPerPerson,,,,,,,,
Spain,26604.223197,2.324009e+07,5.381615e+06,2.755681e+06,11.857444,46505963.0,1.237255e+12,2016.0,0.044570
Denmark,55962.737675,1.384573e+07,3.160679e+06,1.531853e+06,11.063725,5484000.0,3.068997e+11,2016.0,0.019770
Norway,75621.803643,6.746575e+06,1.530765e+06,7.417394e+05,10.994310,4907000.0,3.710762e+11,2016.0,0.014539
Austria,47629.493132,1.912670e+07,4.379840e+06,2.102277e+06,10.991322,8205000.0,3.908000e+11,2016.0,0.023077
United Kingdom,42469.360217,2.675621e+07,6.239173e+06,2.937501e+06,10.978764,62348447.0,2.647899e+12,2016.0,0.025851
Czech Republic,18643.097071,1.317355e+07,3.047783e+06,1.442659e+06,10.951180,10476000.0,1.953051e+11,2016.0,0.058741
Portugal,19186.642742,1.018454e+07,2.371030e+06,1.088609e+06,10.688841,10676000.0,2.048366e+11,2016.0,0.055710
Poland,12243.231395,1.554768e+07,3.701037e+06,1.655277e+06,10.646457,38500000.0,4.713644e+11,2016.0,0.086958
Belgium,44982.765531,1.560840e+07,3.564655e+06,1.654493e+06,10.600021,10403000.0,4.679557e+11,2016.0,0.023565


In [111]:
sales_all[['name','Country Name']].groupby(by='name').count()

,Country Name
name,
Austria,24
Belgium,24
Czech Republic,24
Denmark,24
Finland,23
France,24
Germany,24
Ireland,23
Italy,24


In [21]:
bestRegions['Regions']= sales_all['Regions'].apply(lambda x: 'Slovak Republic' if x == "Slovakia" else x)

In [22]:
bestRegions.rename(columns={'Value':'GDP'}, inplace=True)

In [24]:
bestRegions['gdpPerPerson'] = sales_all['GDP'] / sales_all['population']

In [25]:
sales_all["GrossMargin"] = sales_all['GP']/sales_all['Turnover'] * 100

In [26]:
sales_all.groupby("Regions").sum()[sales_all.groupby("Regions").sum().index == 'Austria']

,index,Units,Cost of Sales,Turnover,Gross Profit,GP,population,GDP,Year,gdpPerPerson,GrossMargin
Regions,,,,,,,,,,,
Austria,2323382,4.379840e+06,1.702442e+07,1.912670e+07,0.0,2.102277e+06,196920000,9.379200e+12,48384.0,1.143108e+06,274.913647


In [27]:
#Experimental
sales_all = sales_all.merge(bestRegions[["Regions","GrossMargin"]], on='Regions', how='left', suffixes=("_perProduct","_perRegion") )

## Visualisation

In [28]:
sales_all.drop(columns=['Gross Profit'], inplace=True)

In [29]:
sales_all

,index,Products,Regions,Months,Years,Versions,Units,Cost of Sales,Turnover,GP,name,population,ISO alpha 3,Country Name,Country Code,GDP,Year,gdpPerPerson,GrossMargin_perProduct,GrossMargin_perRegion
0,456,Desktop High XL,Austria,Year,All Years,Actual,109278.262959,460323.862194,508045.576864,47721.714670,Austria,8205000,AUT,Austria,AUT,3.908000e+11,2016.0,47629.493132,9.393196,10.864676
1,896,Desktop High XL,Belgium,Year,All Years,Actual,106958.760972,407502.471960,460535.225230,53032.753270,Belgium,10403000,BEL,Belgium,BEL,4.679557e+11,2016.0,44982.765531,11.515461,10.419913
2,1364,Desktop High XL,Czech Republic,Year,All Years,Actual,121148.795766,480770.076562,525175.746775,44405.670213,Czech Republic,10476000,CZE,Czech Republic,CZE,1.953051e+11,2016.0,18643.097071,8.455392,10.145496
3,1805,Desktop High XL,Denmark,Year,All Years,Actual,107198.454207,396043.831442,481027.813045,84983.981602,Denmark,5484000,DNK,Denmark,DNK,3.068997e+11,2016.0,55962.737675,17.667166,11.326214
4,2270,Desktop High XL,Finland,Year,All Years,Actual,73259.531359,258835.852800,284082.430762,25246.577962,Finland,5244000,FIN,Finland,FIN,2.385029e+11,2016.0,45481.102272,8.887061,10.266100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,199329,Wireless Mouse XT,Slovak Republic,Year,All Years,Actual,5387.273289,19216.558658,21095.688694,1879.130036,Slovakia,5455000,SVK,NaN,NaN,NaN,NaN,NaN,8.907650,NaN
450,199754,Wireless Mouse XT,Spain,Year,All Years,Actual,21456.833016,107085.280129,118375.923615,11290.643485,Spain,46505963,ESP,Spain,ESP,1.237255e+12,2016.0,26604.223197,9.537956,12.064452
451,200221,Wireless Mouse XT,Sweden,Year,All Years,Actual,13920.767031,51454.468600,55827.664624,4373.196024,Sweden,9045000,SWE,Sweden,SWE,5.144600e+11,2016.0,56877.830050,7.833385,8.700623
452,200688,Wireless Mouse XT,Switzerland,Year,All Years,Actual,21360.550984,73932.227753,80427.458253,6495.230500,Switzerland,7581000,CHE,Switzerland,CHE,6.688513e+11,2016.0,88227.317800,8.075887,9.159550


In [147]:
def explScatter(source, title="Profit Exploration", x='Turnover', y='GP', leg='Products', size='gdpPerPerson'):

    selection = alt.selection_multi(fields=leg.split(), bind='legend')

    # Get selectable Regions Legend bind='legend'
    #selection = alt.selection_interval(fields=[['Products','Regions']], bind='legend') # Get selectable Regions Legend bind='legend'

    scatter = alt.Chart(source).mark_circle().encode(
        x=alt.X(x, sort='-y'),
        y=alt.Y(y, scale=alt.Scale(type='log')),
        color=alt.Color(f'{leg}', sort='-size'),
        size=alt.Size(f'{size}:Q', bin=True),
        tooltip=['Regions','Products', 'GrossMargin_perProduct','GrossMargin_perRegion'],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
        ).add_selection(selection).interactive()



    return (scatter).properties(title=title, width=900, height=600)

In [182]:
sales_all.groupby("Products").sum()

,index,Units,Cost of Sales,Turnover,GP,population,GDP,Year,gdpPerPerson,GrossMargin_perProduct,GrossMargin_perRegion
Products,,,,,,,,,,,
Desktop High XL,87152,1.969977e+06,7.651184e+06,8.609179e+06,9.579943e+05,451878254,1.665805e+13,36288.0,876661.528938,204.622098,181.549612
Desktop High XQ,252366,1.462116e+06,5.645716e+06,6.331598e+06,6.858822e+05,451878254,1.665805e+13,36288.0,876661.528938,218.807079,181.549612
Desktop L,416788,4.490466e+06,1.688419e+07,1.898460e+07,2.100413e+06,451878254,1.665805e+13,36288.0,876661.528938,207.867206,181.549612
Desktop Pro,583146,4.151034e+06,1.564791e+07,1.785606e+07,2.208151e+06,451878254,1.665805e+13,36288.0,876661.528938,208.470702,181.549612
Desktop Pro XL,747047,2.948819e+06,1.154470e+07,1.281641e+07,1.271712e+06,451878254,1.665805e+13,36288.0,876661.528938,183.473798,181.549612
Keyboard GT,804098,4.651626e+05,1.796927e+06,2.016820e+06,2.198924e+05,442011337,1.611472e+13,32256.0,765243.915044,190.360249,161.541953
Keyboard XX,1041209,1.634231e+05,6.180421e+05,6.822705e+05,6.422840e+04,451878254,1.665805e+13,36288.0,876661.528938,177.516093,181.549612
Laptop Mouse S,1183633,4.601118e+05,1.809284e+06,1.997140e+06,1.878564e+05,451878254,1.665805e+13,36288.0,876661.528938,182.381891,181.549612
Notebook GT,1337964,5.617414e+06,2.174949e+07,2.437935e+07,2.629854e+06,451878254,1.665805e+13,36288.0,876661.528938,203.864899,181.549612


In [149]:
explScatter(sales_all, x='Regions', y='GrossMargin_perProduct', leg='Products', size='GP')

alt.Chart(...)

In [103]:
def heatmap(source, x='Products', y='Regions', color='Cost of Sales'):

    heatmap = alt.Chart(sales_all).mark_rect().encode(
        x=alt.X(f'{x}'),
        y=alt.Y(f'{y}',sort='-color'),
        color=f'{color}:Q'
    ).interactive()
    return heatmap.properties(width=900, height=700)

In [40]:
bestRegion

NameError: name 'bestRegion' is not defined

In [150]:
heatmap(bestRegions, color="a")

alt.Chart(...)

In [309]:
sales_all.head()

,index,Products,Regions,Months,Years,Versions,Units,Cost of Sales,Turnover,Gross Profit,GP,name,population,ISO alpha 3,Country Name,Country Code,Value,Year,gdpPerPerson
0,456,Desktop High XL,Austria,Year,All Years,Actual,109278.262959,460323.862194,508045.576864,0.0,47721.714670,Austria,8205000,AUT,Austria,AUT,3.908000e+11,2016.0,47629.493132
1,896,Desktop High XL,Belgium,Year,All Years,Actual,106958.760972,407502.471960,460535.225230,0.0,53032.753270,Belgium,10403000,BEL,Belgium,BEL,4.679557e+11,2016.0,44982.765531
2,1364,Desktop High XL,Czech Republic,Year,All Years,Actual,121148.795766,480770.076562,525175.746775,0.0,44405.670213,Czech Republic,10476000,CZE,Czech Republic,CZE,1.953051e+11,2016.0,18643.097071
3,1805,Desktop High XL,Denmark,Year,All Years,Actual,107198.454207,396043.831442,481027.813045,0.0,84983.981602,Denmark,5484000,DNK,Denmark,DNK,3.068997e+11,2016.0,55962.737675
4,2270,Desktop High XL,Finland,Year,All Years,Actual,73259.531359,258835.852800,284082.430762,0.0,25246.577962,Finland,5244000,FIN,Finland,FIN,2.385029e+11,2016.0,45481.102272


In [176]:
def histo(source, x='GP'):
    histo = alt.Chart(source).mark_bar().encode(
        x=alt.X(f'{x}:Q', bin=True),
        y='count():Q'
    ).properties(
        width=800,
        height=500)
    return histo

In [192]:
histo(bestRegions, x=['GrossMargin'])

alt.Chart(...)

In [286]:

barchart = alt.Chart(bestRegions, background='transparent').mark_bar().encode(
    x=alt.X('Turnover', axis=alt.Axis(gridColor='#6BCCCC', title='Turnover', labelColor='#6BCCCC', labelFontSize=32, titleFontSize=25, titleColor='#fff')),
    y=alt.Y('Regions:N', axis=alt.Axis(gridColor='#6BCCCC', title='Regions', labelColor='#6BCCCC', labelFontSize=32, titleFontSize=25, titleColor='#fff'), sort='-x'),
    color=alt.Color('GrossMargin', scale=alt.Scale(scheme='magma'), legend=alt.Legend(title='Gross Margin', orient='bottom-right', labelColor='white', labelFontSize=25, titleColor='white', titleFontSize=23)),
).properties(width=800, height=900)
barchart

alt.Chart(...)

In [294]:

barchart = alt.Chart(bestProducts, background='transparent').mark_bar().encode(
    x=alt.X('Turnover', axis=alt.Axis(gridColor='#148A15', title='Turnover', labelColor='#fff', labelFontSize=25, titleFontSize=25, titleColor='#fff')),
    y=alt.Y('Products:N', axis=alt.Axis(gridColor='#148A15', title='Regions', labelColor='#fff', labelFontSize=25, titleFontSize=25, titleColor='#fff'), sort='-x'),
    color=alt.Color('GrossMargin', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(title='Gross Margin', orient='bottom-right', labelColor='white', labelFontSize=25, titleColor='white', titleFontSize=23)),
).properties(width=800, height=900)
barchart

alt.Chart(...)

In [677]:
from turtle import width
from numpy import zeros
from sqlalchemy import column


bar = alt.Chart(regions_gdp, background='transparent').mark_bar().encode(
    x=alt.X('GrossMargin:N',bin=True, axis=alt.Axis(gridColor='#C0B8BF', title='Gross Margin', labelColor='#fff', labelFontSize=25, titleFontSize=25, titleColor='#fff')),
    y=alt.Y('sum(GDP)', axis=alt.Axis(grid=False, title='GDP', labels=False, labelColor='#fff', labelFontSize=25, titleFontSize=25, titleColor='#fff'), sort='-x'),
    color=alt.Color('Regions:N', scale=alt.Scale(scheme='purples'), legend=alt.Legend(title='Region by GDP', orient='top-right', labelColor='#fff', labelFontSize=27, titleColor='#fff', titleFontSize=20), sort='-y'),
    
).configure_view(
    strokeWidth=0
).properties(width=800, height=900)
bar

alt.Chart(...)

In [534]:
barchart(bestRegions, labels=True, x="Units", y="Products")

TypeError: 'Chart' object is not callable

In [153]:
bestRegions.head(0)

,Regions,Turnover,Units,GP,GrossMargin


In [214]:
from turtle import bgcolor


selection = alt.selection_multi(fields=['Products'], bind='legend')


charts = alt.Chart(bestProducts).mark_circle().encode(
    x = alt.X(alt.repeat("column"), type='quantitative', axis=alt.Axis()),
    y = alt.Y(alt.repeat("row"), type='quantitative'),
    color='Products:N',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    tooltip=['Products','Turnover', 'Units','GP', 'GrossMargin'],


    
).properties(
    width=150,
    height=150,
).repeat(
    row=['Turnover', 'Units','GP','GrossMargin'],
    column=['GrossMargin', 'GP', 'Units','Turnover']
).add_selection(selection).interactive()

charts

alt.RepeatChart(...)

In [219]:
bestProducts

,Products,Turnover,Units,GP,GrossMargin
0,TFT Monitor XA,3.142961e+06,1.572798e+07,6.278529e+06,8.906986
1,Notebook SX,2.155471e+06,1.228190e+07,4.690695e+06,8.545559
2,TFT Monitor TL,1.820210e+06,1.177017e+07,4.181357e+06,8.268900
3,Notebook GT,1.443416e+06,1.101928e+07,5.316866e+06,10.638874
4,Notebook LXC,1.009887e+06,8.675964e+06,4.602160e+06,11.652616
5,Desktop L,8.833298e+05,8.845371e+06,4.223122e+06,10.891930
6,Desktop Pro,9.471737e+05,7.693622e+06,4.396233e+06,12.565610
7,TFT Monitor XP,1.301432e+06,5.482309e+06,2.423950e+06,9.292741
8,Desktop Pro XL,1.068712e+06,5.232114e+06,1.860802e+06,7.742992
9,Server Power XC,8.421613e+05,3.804030e+06,2.004570e+06,11.067486


In [220]:
alt.Chart(bestProducts).mark_bar().encode(
            x=alt.X('Products', sort='-y'),
            y=alt.Y(alt.repeat("column"), type='quantitative'
                    ).repeat(
                        column=['Turnover', 'GP', 'Units','GrossMargin']
                    )
            )

(alt.RepeatChart(...),)

# TimeSeries

In [15]:
monthSales['Month'] = monthSales['Datetime'].apply(lambda x: x.month)

In [16]:
monthSales

,index,Products,Regions,Months,Years,Versions,Units,Cost of Sales,Turnover,Gross Profit,GP,Datetime,Month
0,0,Desktop High XL,Austria,Apr,2014,Actual,4730.030000,20156.200000,22900.400000,0.0,2744.200000,2014-04-01,4
4,4,Desktop High XL,Austria,Apr,2015,Actual,4966.531500,21164.010000,24045.420000,0.0,2881.410000,2015-04-01,4
8,8,Desktop High XL,Austria,Apr,2016,Actual,5463.184650,24338.611500,27652.233000,0.0,3313.621500,2016-04-01,4
12,12,Desktop High XL,Austria,Apr,2017,Actual,943.839578,4879.568358,4592.102335,0.0,-287.466023,2017-04-01,4
16,16,Desktop High XL,Austria,Apr,2018,Actual,6428.376408,26714.059982,30967.403910,0.0,4253.343928,2018-04-01,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106196,200928,Wireless Mouse XT,United Kingdom,Oct,2016,Actual,328.352581,4845.468075,5344.757250,0.0,499.289175,2016-10-01,10
106200,200932,Wireless Mouse XT,United Kingdom,Oct,2017,Actual,408.747760,1965.544292,2027.341778,0.0,61.797486,2017-10-01,10
106204,200936,Wireless Mouse XT,United Kingdom,Oct,2018,Actual,420.601445,0.000000,0.000000,0.0,0.000000,2018-10-01,10
106220,201064,Wireless Mouse XT,United Kingdom,Sep,2017,Actual,865.837715,3437.812864,3997.702120,0.0,559.889256,2017-09-01,9


In [59]:
productsPerMonth = monthSales.groupby(['Month','Products']).mean().reset_index()

In [69]:
productsPerMonth['MonthName'] = pd.to_datetime(productsPerMonth['Month'], format="%M").dt.strftime('%b')

In [70]:
productsPerMonth

,Month,Products,index,Units,Cost of Sales,Turnover,Gross Profit,GP,MonthName
0,1,Desktop High XL,4163.941176,1656.051741,5982.062705,6822.731969,0.0,840.669264,Jan
1,1,Desktop High XQ,13002.333333,1497.271590,5368.514884,6035.061401,0.0,666.546517,Jan
2,1,Desktop L,21593.000000,4177.446446,14253.142867,16729.880902,0.0,2476.738034,Jan
3,1,Desktop Pro,30030.941176,3149.265191,11334.984993,12936.246867,0.0,1601.261874,Jan
4,1,Desktop Pro XL,39222.733333,2661.777544,9691.552062,10773.498278,0.0,1081.946216,Jan
...,...,...,...,...,...,...,...,...,...
259,12,Subnote XK,162553.211268,627.333697,2195.507963,2462.656610,0.0,267.148647,Jan
260,12,TFT Monitor TL,170416.542169,5634.770820,17466.882920,19718.134359,0.0,2251.251439,Jan
261,12,TFT Monitor XA,179465.290698,7189.556329,23870.745104,26995.288195,0.0,3124.543090,Jan
262,12,TFT Monitor XP,188072.219780,2808.045208,10340.076096,11305.084857,0.0,965.008761,Jan


In [92]:
selection = alt.selection_multi(fields=['Products'], bind='legend') 

linechart = alt.Chart(productsPerMonth).mark_area(
    ).encode(
        x=alt.X('Month:O'),
        y=alt.Y('GP:Q', stack=None),
        row=alt.Row('Products:N'),
        color=alt.Color('Products:N', sort='-y', legend=alt.Legend(title='Products')),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    ).add_selection(selection).interactive().properties(
        height=80, width=800, title='Gross Profit per Product'
    )
linechart

alt.Chart(...)

## Database

In [76]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///DB/data.db", echo=False)

In [77]:
df = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})

In [79]:
df.to_sql('testPandas', con=engine)

3